In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.3.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [910 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelea

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-08-24 00:31:09--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.17MB/s    in 0.8s    

2022-08-24 00:31:11 (1.17 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [8]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("TheReason").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [19]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
#sc = SparkContext('local')
#spark = SparkSession(sc)

from pyspark import SparkFiles
# Import data types
from pyspark.sql.types import *

url = "https://s3.amazonaws.com/the.chum.bucket/US_Regressor_sample.csv"
spark.sparkContext.addFile(url)

schema = StructType(
   [StructField('category_id', StringType(), True),
    StructField('views', IntegerType(), True),
    StructField('comments_disabled', StringType(), True),
    StructField('ratings_disabled', StringType(), True),
    StructField('channel_text', StringType(), True),
    StructField('difference_d', IntegerType(), True),
    StructField('difference_h', IntegerType(), True),
    StructField('published_year', DateType(), True),
    StructField('published_month', DateType(), True),
    StructField('trending_year', DateType(), True),
    StructField('trending_month', DateType(), True),
    ]
  )

df = spark.read.option("header", "true").schema(schema).csv(SparkFiles.get(""), header=True)

df.show()

+-----------+--------+-----------------+----------------+--------------------+------------+------------+--------------+---------------+-------------+--------------+
|category_id|   views|comments_disabled|ratings_disabled|        channel_text|difference_d|difference_h|published_year|published_month|trending_year|trending_month|
+-----------+--------+-----------------+----------------+--------------------+------------+------------+--------------+---------------+-------------+--------------+
|         10|35374613|      """False"""|     """False"""|"G-Eazy - No Limi...|        null|        null|    2017-01-01|           null|   2017-01-01|          null|
|         22|  502185|      """False"""|     """False"""|"The Greatest Tax...|        null|        null|    2017-01-01|           null|   2017-01-01|          null|
|         10| 1006123|      """False"""|     """False"""|"Ariana Grande - ...|        null|        null|    2018-01-01|           null|   2018-01-01|          null|
|         

In [20]:
# Store environmental variable
from getpass import getpass
password = getpass('postgres')

# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://the-reason.cklnwcyuqruo.us-east-1.rds.amazonaws.com/The_Reason"
config = {"user":"postgres",
          "password": password,
          "driver":"org.postgresql.Driver"}

postgres··········


In [21]:
# Write trending_df to table in RDS
df.write.jdbc(url=jdbc_url, table='youtube_analysis_us', mode=mode, properties=config)